### Resnet3D Tanpa Transfer Learning

In [1]:
import os
import cv2
import torch
import random
import numpy as np
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision.models.video import r3d_18
from tqdm import tqdm

# --- KONFIGURASI ---
DATA_ROOT = "/run/media/han/HDD RAIHAN/Real Dataset Cheating Comvis/Preprocessing/dataset_clips_split_ready"
SAVE_PATH = "/home/han/Documents/Kuliah/S5/Comvis/Projek/r3d_18_cheating_best_without_TL.pth"
CLIP_LEN = 32
RESIZE = 112
BATCH_SIZE = 8
LR = 0.0001
EPOCHS = 100
PATIENCE = 7
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Device: {DEVICE}")

# --- 1. DATASET CLASS ---
class VideoClipDataset(Dataset):
    def __init__(self, root, clip_len=32, resize=112):
        self.root = root
        self.clip_len = clip_len
        self.resize = resize
        self.samples = []
        
        if os.path.exists(root):
            self.classes = sorted(os.listdir(root))
            self.class_to_idx = {c: i for i, c in enumerate(self.classes)}
            for cls in self.classes:
                cls_dir = os.path.join(root, cls)
                if not os.path.isdir(cls_dir): continue
                for fname in os.listdir(cls_dir):
                    if fname.endswith(".mp4"):
                        self.samples.append((os.path.join(cls_dir, fname), self.class_to_idx[cls]))
            print(f"[INFO] Loaded {len(self.samples)} clips from {root}")
        else:
            print(f"[ERROR] Root not found: {root}")
            self.classes = []

    def __len__(self):
        return len(self.samples)

    def _sample_frames(self, cap, total_frames):
        frames = []
        needed = self.clip_len
        if total_frames <= 0: return torch.zeros((3, needed, self.resize, self.resize), dtype=torch.float32)
        
        start = random.randint(0, total_frames - needed) if total_frames > needed else 0
        cap.set(cv2.CAP_PROP_POS_FRAMES, start)
        
        count = 0
        while count < needed:
            ret, frame = cap.read()
            if not ret: # Padding jika video habis/rusak
                frames.append(np.zeros((self.resize, self.resize, 3), dtype=np.uint8))
                count += 1
                continue
            try:
                frame = cv2.resize(frame, (self.resize, self.resize))
                frames.append(frame[:, :, ::-1]) # BGR to RGB
                count += 1
            except: continue
            
        frames = np.stack(frames).transpose(3, 0, 1, 2) / 255.0
        return torch.tensor(frames, dtype=torch.float32)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        cap = cv2.VideoCapture(path)
        clip = self._sample_frames(cap, int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
        cap.release()
        return clip, torch.tensor(label)

# --- 2. SETUP ---
train_set = VideoClipDataset(os.path.join(DATA_ROOT, "train"), CLIP_LEN, RESIZE)
val_set = VideoClipDataset(os.path.join(DATA_ROOT, "val"), CLIP_LEN, RESIZE)

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# --- 3. MODEL (PURE RESNET / NO TRANSFER LEARNING) ---
print("Initializing Pure ResNet3D-18 (Random Weights)...")
model = r3d_18(weights=None) # <--- MURNI (DARI NOL)
model.fc = nn.Linear(model.fc.in_features, 2)
model = model.to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
scaler = torch.amp.GradScaler('cuda')

# --- 4. TRAINING HELPERS ---
class EarlyStopping:
    def __init__(self, patience=5, path='checkpoint.pth'):
        self.patience = patience
        self.path = path
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.inf

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score:
            self.counter += 1
            print(f'   -> EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience: self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        print(f'   -> Val loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}). Saving...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

def train_epoch():
    model.train()
    total_loss, correct, total = 0, 0, 0
    for clips, labels in tqdm(train_loader, desc="Train"):
        clips, labels = clips.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        with torch.amp.autocast('cuda'):
            outputs = model(clips)
            loss = criterion(outputs, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        total_loss += loss.item() * clips.size(0)
        _, pred = outputs.max(1)
        correct += pred.eq(labels).sum().item()
        total += labels.size(0)
    return total_loss/total, correct/total

def val_epoch():
    model.eval()
    total_loss, correct, total = 0, 0, 0
    with torch.no_grad():
        for clips, labels in tqdm(val_loader, desc="Val"):
            clips, labels = clips.to(DEVICE), labels.to(DEVICE)
            outputs = model(clips)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * clips.size(0)
            _, pred = outputs.max(1)
            correct += pred.eq(labels).sum().item()
            total += labels.size(0)
    return total_loss/total, correct/total

# --- 5. EXECUTION ---
history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
early_stopping = EarlyStopping(patience=PATIENCE, path=SAVE_PATH)

print("🚀 Mulai Training...")
for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch+1}/{EPOCHS}")
    tl, ta = train_epoch()
    vl, va = val_epoch()
    
    history['train_loss'].append(tl); history['train_acc'].append(ta)
    history['val_loss'].append(vl); history['val_acc'].append(va)
    
    print(f"Train Loss: {tl:.4f} | Acc: {ta:.4f}")
    print(f"Val   Loss: {vl:.4f} | Acc: {va:.4f}")
    
    early_stopping(vl, model)
    if early_stopping.early_stop:
        print("🛑 Early stopping triggered!")
        break

print(f"✅ Selesai. Model disimpan di: {SAVE_PATH}")

Device: cuda
[INFO] Loaded 1865 clips from /run/media/han/HDD RAIHAN/Real Dataset Cheating Comvis/Preprocessing/dataset_clips_split_ready/train
[INFO] Loaded 392 clips from /run/media/han/HDD RAIHAN/Real Dataset Cheating Comvis/Preprocessing/dataset_clips_split_ready/val
Initializing Pure ResNet3D-18 (Random Weights)...
🚀 Mulai Training...

Epoch 1/100


Val: 100%|██████████| 49/49 [00:13<00:00,  3.60it/s]


Train Loss: 0.5459 | Acc: 0.7351
Val   Loss: 0.4591 | Acc: 0.8112
   -> Val loss decreased (inf --> 0.459139). Saving...

Epoch 2/100


Train:  35%|███▌      | 83/234 [01:40<03:02,  1.21s/it]


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

# Pastikan cell training sudah selesai dijalankan agar variabel 'history' ada isinya.

plt.figure(figsize=(12, 5))

# Grafik Loss
plt.subplot(1, 2, 1)
plt.plot(history['train_loss'], label='Train Loss', color='blue')
plt.plot(history['val_loss'], label='Val Loss', color='red')
plt.title('Training & Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

# Grafik Akurasi
plt.subplot(1, 2, 2)
plt.plot(history['train_acc'], label='Train Acc', color='blue')
plt.plot(history['val_acc'], label='Val Acc', color='green')
plt.title('Training & Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
import torch
import torch.nn as nn
from torchvision.models.video import r3d_18
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import os

# --- KONFIGURASI TEST ---
# Pastikan path ini sama dengan yang di Cell 1
DATA_ROOT = "/run/media/han/HDD RAIHAN/Real Dataset Cheating Comvis/Preprocessing/dataset_clips_split_ready"
MODEL_PATH = "/home/han/Documents/Kuliah/S5/Comvis/Projek/r3d_18_cheating_best_without_TL.pth"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Setup Dataset Test (Menggunakan Class dari Cell 1)
test_set = VideoClipDataset(os.path.join(DATA_ROOT, "test"), clip_len=32, resize=112)
test_loader = DataLoader(test_set, batch_size=8, shuffle=False, num_workers=2)
class_names = test_set.classes

print(f"Testing pada {len(test_set)} klip video...")

if len(test_set) > 0:
    # 1. INISIALISASI MODEL (STRUKTUR HARUS SAMA PERSIS DENGAN TRAIN)
    # Karena train pakai Pure Resnet (weights=None), di sini juga harus sama.
    model_test = r3d_18(weights=None) 
    model_test.fc = nn.Linear(model_test.fc.in_features, 2)
    
    # 2. LOAD WEIGHTS
    print(f"📥 Memuat model dari: {MODEL_PATH}")
    checkpoint = torch.load(MODEL_PATH)
    model_test.load_state_dict(checkpoint)
    model_test.to(DEVICE)
    model_test.eval()

    # 3. PREDIKSI
    all_preds = []
    all_labels = []
    
    print("🔄 Menjalankan prediksi...")
    with torch.no_grad():
        for clips, labels in tqdm(test_loader):
            clips = clips.to(DEVICE)
            outputs = model_test(clips)
            _, preds = torch.max(outputs, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())

    # 4. LAPORAN HASIL
    print("\n" + "="*40)
    print("CLASSIFICATION REPORT")
    print("="*40)
    print(classification_report(all_labels, all_preds, target_names=class_names))

    # 5. CONFUSION MATRIX
    plt.figure(figsize=(6, 5))
    cm = confusion_matrix(all_labels, all_preds)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Prediksi')
    plt.ylabel('Asli')
    plt.title('Confusion Matrix (Test Data)')
    plt.show()

else:
    print("⚠️ Folder test kosong atau tidak ditemukan!")